In [9]:
import numpy as np
import torch
import torch.nn as nn

In [10]:
#load data
train_data = np.loadtxt('./train.csv', delimiter=',', dtype=str, skiprows=1)
#label = data
#set_label = set(label)
#print(data)
torch.manual_seed(0)

In [27]:
#load one-hot encoding set
symptom_set = disease_set = []
with open('ohe_symptoms.txt', 'r') as ohe:
    symptom_set = ohe.readlines()
symptom_set = [s[:-1] for s in symptom_set]

with open('ohe_diseases.txt', 'r') as ohe:
    disease_set = ohe.readlines()
disease_set = [d[:-1] for d in disease_set]

In [16]:
#131 possible symptoms
#41 diseases
#print(data)
print(symptom_set)

3


In [11]:
def transform_train_data(X):
    N,d = X.shape

    symptoms = X[:,1:]
    set_symptom = set(symptoms.flatten())
    set_symptom.remove('')
    symptoms = list(set_symptom)

    i = 0
    for s in symptoms:
        if s[0] == ' ':
            symptoms[i] = symptoms[i][1:]
        i += 1

    diseases = X[:,0]
    set_diseases = set(diseases)
    diseases = list(set_diseases)

    trans_input = np.zeros(shape=(N, 131), dtype=int)
    trans_label = np.zeros(shape=(N,1), dtype=int)
    for i in range(N):
        for sym in X[i,1:]:
            if sym != '':
                s = sym
                if s[0] == ' ':
                    s = s[1:]
                trans_input[i,symptoms.index(s)] = 1
        trans_label[i,0] = diseases.index(X[i,0])
    return trans_input, trans_label, symptoms, diseases

In [12]:
#one-hot encode data
t_data, t_label, symp, dis = transform_train_data(train_data)
train_data, train_label = t_data, t_label

#validation/train split
#valid_rows = np.random.choice(len(t_data),size=40, replace=False)
#valid_data, valid_label = t_data[valid_rows,:], t_label[valid_rows,:]
#train_data, train_label = np.delete(t_data, valid_rows, axis=0), np.delete(t_label, valid_rows, axis=0)

In [24]:
#save one-hot encoding
with open('ohe_symptoms.txt', 'w') as ohe:
    ohe.writelines('%s\n' % s for s in symp)
with open('ohe_diseases.txt', 'w') as ohe:
    ohe.writelines('%s\n' % d for d in dis)
#np.savetxt('ohe_symptoms.csv', np.array(symp), fmt='%s')
#np.savetxt('ohe_diseases.csv', np.array(dis), fmt='%s')

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Running on GPU')
else:
    device = torch.device('cpu')
    print('Running on CPU')

Running on CPU


In [113]:
'''
class NN(nn.Module):
    def __init__(self, num_dis, num_symp):
        super(NN, self).__init__()
        self.num_dis = num_dis
        self.num_symp = num_symp
        self.ll1 = nn.Linear(num_symp, 512)
        self.ll2 = nn.Linear(512, 512)
        self.ll3 = nn.Linear(512, 256)
        self.ll4 = nn.Linear(256, 512)
        self.ll5 = nn.Linear(512, 256)
        self.ll6 = nn.Linear(256, 128)
        self.ll5 = nn.Linear(128, num_dis)
        
    def forward(self, x):
        x = F.relu(self.ll1(x))
        x = F.relu(self.ll2(x))
        x = F.relu(self.ll3(x))
        x = F.relu(self.ll4(x))
        x = F.relu(self.ll5(x))
        x = F.relu(self.ll6(x))
        x = self.ll7(x)
        return x
'''

In [14]:
#model
from model import CS145_NN
model_d = CS145_NN()
model_d.to(device)

CS145_NN(
  (ll1): Linear(in_features=131, out_features=512, bias=True)
  (ll2): Linear(in_features=512, out_features=128, bias=True)
  (ll5): Linear(in_features=128, out_features=41, bias=True)
  (relu): ReLU()
)

In [15]:
#Loss Function
loss_f = nn.CrossEntropyLoss()
#Optimizer
optim = torch.optim.Adam(model_d.parameters(), lr=0.0025)

In [16]:
def train(input, label, model, batch_size, loss_func, optimizer, device):
    input, label = torch.tensor(input).to(device).float(), torch.tensor(label).to(device).long()
    model = model.to(device)

    losses = []
    model.train()
    n_batch = int(len(label)/batch_size)
    if n_batch * batch_size < len(label):
        n_batch += 1

    
    for b in range(n_batch):
        optimizer.zero_grad()
        inp = input[b*batch_size:(b+1)*batch_size]
        lab = label[b*batch_size:(b+1)*batch_size]
        pred = model(inp)
        loss = loss_func(pred, lab.flatten())
        loss.backward()
        losses.append(loss)
        optimizer.step()
    print(f'End of Epoch Loss: {round((sum(losses)/len(input)).item(), 3)}')



In [30]:
#validation
def test(input, label, model, device):
    model = model.to(device)
    input, label = torch.tensor(input).to(device).float(), torch.tensor(label).to(device).long()
    model.eval()
    prediction = model(input).argmax(axis=1)
    #print(label)
    #print(prediction)
    correct = (prediction == label.flatten()).sum().item()
    print(f'Accuracy: {correct}')
    

In [17]:
#Train/Test Script
for epoch in range(1,51):
    print('Epoch: ', epoch)
    train(train_data, train_label, model_d, 32, loss_f, optim, device)
    #test(valid_data, valid_label, model_d, device)

Epoch:  1
End of Epoch Loss: 0.112
Epoch:  2
End of Epoch Loss: 0.062
Epoch:  3
End of Epoch Loss: 0.016
Epoch:  4
End of Epoch Loss: 0.003
Epoch:  5
End of Epoch Loss: 0.001
Epoch:  6
End of Epoch Loss: 0.0
Epoch:  7
End of Epoch Loss: 0.0
Epoch:  8
End of Epoch Loss: 0.0
Epoch:  9
End of Epoch Loss: 0.0
Epoch:  10
End of Epoch Loss: 0.0
Epoch:  11
End of Epoch Loss: 0.0
Epoch:  12
End of Epoch Loss: 0.0
Epoch:  13
End of Epoch Loss: 0.0
Epoch:  14
End of Epoch Loss: 0.0
Epoch:  15
End of Epoch Loss: 0.0
Epoch:  16
End of Epoch Loss: 0.0
Epoch:  17
End of Epoch Loss: 0.0
Epoch:  18
End of Epoch Loss: 0.0
Epoch:  19
End of Epoch Loss: 0.0
Epoch:  20
End of Epoch Loss: 0.0
Epoch:  21
End of Epoch Loss: 0.0
Epoch:  22
End of Epoch Loss: 0.0
Epoch:  23
End of Epoch Loss: 0.0
Epoch:  24
End of Epoch Loss: 0.0
Epoch:  25
End of Epoch Loss: 0.0
Epoch:  26
End of Epoch Loss: 0.0
Epoch:  27
End of Epoch Loss: 0.0
Epoch:  28
End of Epoch Loss: 0.0
Epoch:  29
End of Epoch Loss: 0.0
Epoch:  30
En

In [18]:
def predict(input, model, device):
    model = model.to(device)
    input = torch.tensor(input).to(device).float()
    model.eval()
    prediction = model(input).argmax(axis=1)
    return prediction

In [19]:
def transform_test_data(X, symptom_set):
    N,d = X.shape
    symptoms = X[:,1:]
    test_input = np.zeros(shape=(N,131), dtype=int)
    #print(symptom_set)
    for i in range(N):
        for sym in symptoms[i,:]:
            if sym != '':
                s = sym
                if s[0] == ' ':
                    s = s[1:]
                test_input[i,symptom_set.index(s)] = 1
    return test_input

In [20]:
def transform_test_label(y, disease_set):
    N, = y.shape
    to_csv = np.empty(shape=(N+1,2),dtype=object)
    to_csv[0] = ['ID','Disease']
    for i in range(N):
        to_csv[i+1] = [str(i+1), disease_set[y[i]]]
        
    return to_csv
    

In [21]:
test_data = np.loadtxt('./test.csv', delimiter=',', dtype=str, skiprows=1)
test_input = transform_test_data(test_data, symp)
test_pred = predict(test_input, model_d, device)
csv_arr = transform_test_label(test_pred, dis)

In [22]:
#save csv
np.savetxt('s21.csv',csv_arr, delimiter=',', fmt='%s')

In [23]:
#save weights
torch.save(model_d, 'cs145_m21.pt')
torch.save(model_d.state_dict(), 'cs145_s21.pt')
#ms = torch.jit.script(model_d)
#ms.save('cs145_ms20.pt')

In [18]:
#load weights
model_d.load_state_dict(torch.load("cs145_s19.pt", map_location=device))
#model_d = torch.jit.load('cs145_ms20.pt')
model_d.eval()


RuntimeError: Error(s) in loading state_dict for RecursiveScriptModule:
	size mismatch for ll2.weight: copying a param with shape torch.Size([128, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for ll2.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for ll5.weight: copying a param with shape torch.Size([41, 128]) from checkpoint, the shape in current model is torch.Size([41, 256]).

In [24]:
for param in model_d.parameters():
        print(param.data)

tensor([[-0.0374,  0.1373, -0.0719,  ...,  0.0246, -0.0716,  0.0705],
        [-0.0177, -0.1162, -0.0045,  ..., -0.0850, -0.0866,  0.0343],
        [ 0.1321,  0.0481, -0.0331,  ...,  0.0910, -0.0877, -0.1285],
        ...,
        [-0.1032, -0.0483, -0.1489,  ...,  0.0614, -0.0406,  0.0097],
        [-0.1133,  0.0761,  0.0632,  ...,  0.0438,  0.0860,  0.0048],
        [-0.1348, -0.0324,  0.0314,  ...,  0.0782,  0.1394,  0.0767]])
tensor([ 1.0674e-01,  9.0635e-02,  1.2956e-01,  8.5871e-02,  4.2702e-02,
         9.8036e-02,  9.3130e-02,  1.8881e-01,  7.3198e-02,  1.3612e-01,
         9.0414e-04, -1.5010e-03,  1.5433e-01,  1.4147e-01,  1.3897e-01,
         1.8595e-01,  1.1150e-01,  3.7593e-02,  1.6711e-02,  6.2511e-02,
         1.6203e-01,  1.1519e-01,  1.4757e-01, -1.1442e-02,  1.2283e-01,
         1.4929e-01,  3.5682e-02,  1.2927e-01,  1.4095e-01,  7.9222e-02,
         1.5836e-01,  5.3494e-02,  1.1332e-01,  7.2772e-02,  1.0601e-01,
         5.8297e-02,  1.0291e-01,  1.8111e-01,  1.3357e

In [15]:
for param in model_d.parameters():
        print(param.data)

tensor([[-0.0012, -0.0345,  0.0897,  ...,  0.1561, -0.1255,  0.1192],
        [ 0.0227,  0.0353, -0.0450,  ..., -0.0114, -0.0044,  0.0367],
        [-0.1016,  0.0405,  0.0441,  ..., -0.0188,  0.0172,  0.1058],
        ...,
        [-0.0671,  0.0205,  0.1105,  ...,  0.0521,  0.0090,  0.0586],
        [-0.0170, -0.1160, -0.0456,  ..., -0.1243, -0.0762,  0.1202],
        [-0.0392, -0.0541,  0.0588,  ..., -0.1138,  0.0201,  0.0385]])
tensor([ 0.1169,  0.1526,  0.1007,  0.1636,  0.1883,  0.1289,  0.1169, -0.0078,
         0.0728,  0.0975,  0.1432,  0.0622,  0.1452,  0.0830,  0.1332,  0.1127,
         0.1435,  0.0579,  0.0296,  0.1865,  0.0349,  0.0569,  0.0706,  0.0864,
         0.0913,  0.0552,  0.0183,  0.0101,  0.0208,  0.0158,  0.0117,  0.0986,
         0.0920,  0.0945,  0.0491,  0.0562,  0.1469,  0.0291,  0.0952,  0.1503,
        -0.0235,  0.0604,  0.0016,  0.0717,  0.1273,  0.0435,  0.0495, -0.0062,
        -0.0103,  0.0377,  0.1608,  0.0493,  0.1070,  0.0577,  0.0981,  0.1493,
      